<a href="https://colab.research.google.com/github/Ashish-17CSE/Hate_Speech_Detection/blob/main/Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload Data


In [ ]:
# Upload the train.csv and val.csv file from your local drive
from google.colab import files
uploaded = files.upload()

Saving dev_3k_split_conll.txt to dev_3k_split_conll.txt
Saving Hindi_test_unalbelled_conll_updated.txt to Hindi_test_unalbelled_conll_updated.txt
Saving test_labels_hinglish.txt to test_labels_hinglish.txt
Saving train_14k_split_conll .txt to train_14k_split_conll .txt


In [36]:
dataset = '/content/dev_3k_split_conll.txt'

def read_conll(in_file, lowercase=False, max_example=None):
    data = []
    with open(in_file) as f:
        words, lang = [], []
        id = "";
        label = "";
        for line in f.readlines():
            sp = line.strip().split('\t')
            if sp[0]=="meta" and len(sp)>2:
              vc = {"id":id,"label":label,"words":words,"lang":lang}
              data.append(vc);
              words,lang = [],[]
              id = sp[1]
              label = sp[2]
            elif len(sp)>1:
              words.append(sp[0])
              lang.append(sp[1])
        vc = {"id":id,"label":label,"words":words,"lang":lang}
        data.append(vc);
    return data 

MyData = read_conll(dataset)[1:];
print(MyData)

[{'id': '30258', 'label': 'positive', 'words': ['@', 'prahladspatel', 'modi', 'mantrimandal', 'may', 'samil', 'honay', 'par', 'badhai', 'narmaday', 'har'], 'lang': ['O', 'Eng', 'Eng', 'Eng', 'Eng', 'Hin', 'Hin', 'Hin', 'Hin', 'Hin', 'Hin']}, {'id': '16648', 'label': 'negative', 'words': ['@', 'bkunalraj', '@', 'TajinderBagga', '@', 'NikhilJakhar14', '@', 'JammuPalchhin', '@', 'Shehla', '_', 'Rashid', '@', 'TajinderBagga', 'Tu', 'toh', 'naamakool', 'hai', 'Mare', 'h', '…', 'https', '//', 't', '.', 'co', '/', 'P003zxqUtP'], 'lang': ['O', 'Eng', 'O', 'Eng', 'O', 'Eng', 'O', 'Hin', 'O', 'Hin', 'O', 'Eng', 'O', 'Hin', 'Hin', 'Hin', 'Hin', 'Hin', 'Hin', 'Eng', 'O', 'Eng', 'O', 'Eng', 'O', 'Eng', 'O', 'Eng']}, {'id': '28511', 'label': 'negative', 'words': ['@', 'waglenikhil', 'U', 'saw', 'caste', 'and', 'religion', 'in', 'them', '...', 'nation', 'saw', 'talent', 'and', 'trust', 'in', 'them', '!!', '..', 'problem', 'is', 'tum', 'paida', 'hi', 'ulte', 'hue', 'the', '!!'], 'lang': ['O', 'Hin', '

In [37]:
import pandas as pd

In [38]:
def read_conll(in_file, lowercase=False, max_example=None):
    data = []
    with open(in_file) as f:
        words, lang = [], []
        id = "";
        label = "";
        for line in f.readlines():
            sp = line.strip().split('\t')
            if sp[0]=="meta" and len(sp)>2:
              vc = {"id":id,"label":label,"words":words,"lang":lang}
              data.append(vc);
              words,lang = [],[]
              id = sp[1]
              label = sp[2]
            elif len(sp)>1:
              words.append(sp[0])
              lang.append(sp[1])
        vc = {"id":id,"label":label,"words":words,"lang":lang}
        data.append(vc);
    return data 


In [39]:
#convert to sentances

def read_conll(in_file, lowercase=False, max_example=None):
    data = []
    with open(in_file) as f:
        words = " "
        id = "";
        label = "";
        for line in f.readlines():
            sp = line.strip().split('\t')
            if sp[0]=="meta" and len(sp)>2:
              vc = {"id":id,"label":label,"tweet":words}
              data.append(vc);
              words = " "
              id = sp[1]
              label = sp[2]
            elif len(sp)>1:
              if sp[1]!="O":
                words = words + sp[0] +" " #+"("+sp[1]+") "
        vc = {"id":id,"label":label,"tweet":words}
        data.append(vc);
    return data 

In [44]:
#Merge test labels and tweets
'''
 Me Rajendra hu
 Hin Eng     Hin

 Me Rajendra Hu(Hin)
'''

def read_conll_test(in_file, lowercase=False, max_example=None):
    data = []
    with open(in_file) as f:
        words = " "
        id = "";
        label = "";
        for line in f.readlines():
            print(line)
            sp = line.strip().split('\t')
            if sp[0]=="meta":
              vc = {"id":id,"tweet":words}
              data.append(vc);
              words = " "
              id = sp[1]
            elif len(sp)>1:
              if sp[1]!="O":
                words = words + sp[0] +" " # +"("+sp[1]+") "
        vc = {"id":id,"tweet":words}
        data.append(vc);
    return data 

dataset = "/content/Hindi_test_unalbelled_conll_updated.txt"
test = read_conll_test(dataset)[1:]
test = pd.DataFrame(test)
test.head()

Streaming output truncated to the last 5000 lines.
RSS	Eng

group	Eng

it's	Eng

my	Eng

humble	Eng

sugess	Eng

â€¦	O

https	Eng

//	O

t	Eng

.	O

co	Eng

/	O

xUqe0L5e2y	Eng



meta	11235

RT	Eng

@	O

Kalpesh63520042	Eng

@	O

DrHarkar	Eng

@	O

LaxmiBi87208189	Eng

Hindu	Eng

Sanskruti	Hin

ki	Hin

raksha	Hin

karne	Hin

valo	Hin

ko	Hin

hi	Hin

jhute	Hin

aropo	Eng

me	Hin

fasaya	Hin

jata	Hin

hai	Hin

#	O

WeSupportPunalek	Hin

â€¦	O



meta	13204

Nothing	Eng

funny	Eng

Amardeep	Hin

.	O

We	Hin

are	Eng

not	Eng

suggesting	Eng

all	Eng

the	Eng

writings	Eng

in	Eng

the	Eng

Dasam	Hin

Granth	Hin

are	Eng

unauthentic	Eng

.	O

We	Eng

think	Eng

you	Eng

ma	Eng

â€¦	O

https	Eng

//	O

t	Eng

.	O

co	Hin

/	O

MqGoUopLAn	Hin



meta	9175

@	O

itz	Eng

_	O

faiz	Eng

@	O

PCsTinkerbell	Eng

Sale	Eng

chutiye	Hin

tu	Hin

kuch	Hin

update	Hin

nahi	Hin

rehta	Hin

?	O

Ye	Hin

dekh	Hin

tere	Hin

criminal	Hin

buddhe	Hin

aur	Hin

bharat	Hin

team	Hin

kis	Hin

h	Hin

â€

,id,tweet
0,20803,454dkhan Heisunberg Agr kse ko itni important...
1,20187,logon ko alloo pyaz tomator me toh allah pak ...
2,20953,LambaAlka Wafadaar bane rahane ka nayab tarik...
3,13777,varnishant narendramodi Chup bhosdike He has ...
4,20990,RT HardeepSPuri Praise the Great Giver who gi...


In [45]:
labels = pd.read_csv("/content/test_labels_hinglish.txt")
labels = pd.DataFrame(labels)
labels.columns=["id","label"] 
labels['id']=labels['id'].astype(str)
labels.head()

,id,label
0,20803,neutral
1,20187,neutral
2,20953,neutral
3,13777,negative
4,20990,positive


In [46]:
print(test.dtypes)
print(labels.dtypes)

id       object
tweet    object
dtype: object
id       object
label    object
dtype: object


In [47]:
# test = pd.concat([test, labels], axis = 1)
test = pd.merge(test, labels, how='inner', on = 'id')
test.head()

,id,tweet,label
0,20803,454dkhan Heisunberg Agr kse ko itni important...,neutral
1,20187,logon ko alloo pyaz tomator me toh allah pak ...,neutral
2,20953,LambaAlka Wafadaar bane rahane ka nayab tarik...,neutral
3,13777,varnishant narendramodi Chup bhosdike He has ...,negative
4,20990,RT HardeepSPuri Praise the Great Giver who gi...,positive
